In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#!pip install pytorch-lightning

In [ ]:
import pytorch_lightning as pl
import pandas as pd
import cv2
import os 
from torch import nn
from torch.utils.data import Dataset ,DataLoader, random_split
import numpy as np
import torch
from sklearn.model_selection import train_test_split 
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import torchmetrics
from torchmetrics.functional import accuracy
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from torchvision.utils import make_grid
import math
import torch.nn.functional as F

In [ ]:
pl.seed_everything(7)
PATH_DATASETS = os.environ.get("PATH_DATASETS", "/content/drive/MyDrive/Datasets")
BATCH_SIZE = 256 if torch.cuda.is_available() else 64
CLASS_SIZE = 10
VAL_SIZE = 10000

Global seed set to 7


In [ ]:
class Data(pl.LightningDataModule):
  def __init__(self, data_dir: str=PATH_DATASETS, batch_size: int=BATCH_SIZE, class_size: int=CLASS_SIZE,
               val_size: int=VAL_SIZE):
    super().__init__()
    self.data_dir = data_dir
    self.batch_size = batch_size
    self.class_size = class_size
    self.val_size = val_size
    self.transform = {
        'train': transforms.Compose([
            transforms.Resize((224,224)),
            transforms.RandomHorizontalFlip(p=0.7),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ]),

        'test': transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
    }

    

  def prepare_data(self):
    datasets.CIFAR10(root=self.data_dir, train=True, download=True)
    datasets.CIFAR10(root=self.data_dir, train=False, download=True)

  def setup(self, stage=None):
    if stage == 'fit' or stage is None:
      cifar10_full = datasets.CIFAR10(root=self.data_dir, train=True, transform=self.transform['train'])
      self.train_size = len(cifar10_full) - self.val_size
      self.cifar_train, self.cifar_val = random_split(cifar10_full, [self.train_size, self.val_size])

    if stage == 'test' or stage is None:
      self.cifar_test = datasets.CIFAR10(root=self.data_dir, train=False, transform=self.transform['test'])

  def train_dataloader(self):
    return DataLoader(self.cifar_train, batch_size=32, shuffle=True)

  def val_dataloader(self):
    return DataLoader(self.cifar_val, batch_size=32, shuffle=False)

  def test_dataloader(seld):
    return DataLoader(self.cifar_test, batch_size=32, shuffle=False)

  def visualize(self):
    for images, _ in self.train_dataloader():
      print(f"images.shape: {images.shape}")
      plt.figure(figsize=(16,8))
      plt.axis("off")
      plt.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
      break

In [ ]:
dm = Data()
dm.prepare_data()

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
dm.setup()

In [ ]:
dm.visualize()

In [ ]:
class LitVGGNet(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)

    self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
    self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)

    self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
    self.conv6 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
    self.conv7 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

    self.conv8 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
    self.conv9 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
    self.conv10 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

    self.conv11 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
    self.conv12 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
    self.conv13 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

    self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    self.fc14 = nn.Linear(in_features=7*7*512, out_features=4096)
    self.fc15 = nn.Linear(in_features=4096, out_features=4096)
    self.fc16 = nn.Linear(in_features=4096, out_features=10)

    self.save_hyperparameters()
  
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = self.maxpool(x)
    x = F.relu(self.conv3(x))
    x = F.relu(self.conv4(x))
    x = self.maxpool(x)
    x = F.relu(self.conv5(x))
    x = F.relu(self.conv6(x))
    x = F.relu(self.conv7(x))
    x = self.maxpool(x)
    x = F.relu(self.conv8(x))
    x = F.relu(self.conv9(x))
    x = F.relu(self.conv10(x))
    x = self.maxpool(x)
    x = F.relu(self.conv11(x))
    x = F.relu(self.conv12(x))
    x = F.relu(self.conv13(x))
    x = self.maxpool(x)
    x = x.view(x.size(0), -1)
    x = F.relu(self.fc14(x))
    x = F.dropout(x, 0.5)
    x = F.relu(self.fc15(x))
    x = F.dropout(x, 0.5)
    x = F.relu(self.fc16(x))
    return x


  def training_step(self, batch, batch_idx):
    x, y = batch
    logits = self.forward(x)
    loss = nn.functional.cross_entropy(logits, y)
    self.log('train loss', loss)
    return loss

  def evaluate(self, batch, stage=None):
    x, y = batch
    logits = self.forward(x)
    loss = nn.functional.cross_entropy(logits, y)
    preds = torch.argmax(logits, dim=1)
    acc = accuracy(preds, y)

    if stage:
      self.log(f'{stage}_loss', loss, prog_bar=True)
      self.log(f'{stage}_acc', acc, prog_bar=True)

  def validation_step(self, batch, batch_idx):
    self.evaluate(batch, stage="val")

  def test_step(self, batch, batch_idx):
    self.evaluate(batch, stage="test")

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=1e-4)

In [ ]:
model = LitVGGNet()
trainer = pl.Trainer(max_epochs=10,
                  default_root_dir="/content/drive/MyDrive/Checkpoints", 
                  accelerator="auto",
                  devices=1 if torch.cuda.is_available() else None,
                  callbacks=[LearningRateMonitor(logging_interval="step"),
                                TQDMProgressBar(refresh_rate=10)],
)
trainer.fit(model, dm)